In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
!pwd

/d/PythonProjects/Text-summarizer


In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_size

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        
        create_directories([config.root_dir])
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )
        return model_trainer_config  

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import tensorflow as tf

In [11]:
class ModelTrainer:
    def __init__(self,
                 config:ModelTrainerConfig):
        self.config=config
    
    def train(self):
        
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        pegasus_model=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        data_collector=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=pegasus_model)
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 
 
        
        
        trainer = Trainer(model=pegasus_model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=data_collector,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()
        
        pegasus_model.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-05 10:33:33,719 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-08-05 10:33:33,728 : INFO : common : yaml file: params.yaml loaded successfully]
[2023-08-05 10:33:33,734 : INFO : common : created directory at: artifacts]
[2023-08-05 10:33:33,738 : INFO : common : created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: '>' not supported between instances of 'str' and 'int'